# path search (A/A*, Dynamic programming)
search optimal path in the map

## Optimal path search algorithm without any heuristic function

In [1]:
# -----------
# User Instructions:
# 
# Modify the function search so that it returns
# a table of values called expand. This table
# will keep track of which step each node was
# expanded.
#
# Make sure that the initial cell in the grid 
# you return has the value 0.
# ----------

## map grid
grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0]]

## initial point
init = [0, 0]
## goal point
goal = [len(grid)-1, len(grid[0])-1]
## step cost
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']


In [2]:
def search(grid,init,goal,cost):
    # ----------------------------------------
    # modify code below
    # ----------------------------------------
    
    ## flag grid to check whether the position in the map is explored
    closed_grid = [[0 for i in range(len(grid[0]))] for j in range(len(grid))]
    ## how many steps in the certain grid the robot has explored
    expand_grid = [[-1 for i in range(len(grid[0]))] for j in range(len(grid))]
    ## obstacle positions
    obstacle_list = [[i,j] for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j]==1]
    ## keep track of the actions when doing path searching
    action = [[-1 for row in range(len(grid[0]))] for col in range(len(grid))]
    
    closed_grid[0][0] = 1
    
    x = init[0]
    y = init[1]
    ## total current cost value
    g = 0
    ## open position list which is not explored
    open_list = [[g,x,y]]
    ## two flags for identifying whether to continue to search
    found = False
    resign = False
    
    step = 1
    
    while found is False and resign is False:
        
        if open_list == []:
            resign = True
            print('fail')
            break
        
        current_list = open_list.copy()
        ## return the current minimum cost
        min_cost = min(list(zip(*current_list))[0])
        ## loop for potential next point to explore 
        for next_point in current_list:
            ## select the point with the min cost for the exploration
            if next_point[0] != min_cost:
                continue
            
            current_point = next_point.copy()
            
            g = current_point[0]
            x = current_point[1]
            y = current_point[2]
            
            if [x,y] == goal:
                path = [g,x,y]
                found = True
            ## remove the explored point in the open list
            open_list.remove(current_point)
            
            expand_grid[x][y] += step
            step += 1
            ## loop over all possible movement
            for i, move in enumerate(delta):
                
                temp_g = g + cost
                temp_x = x + move[0]
                temp_y = y + move[1]
                ## check feasibility
                if [temp_x,temp_y] not in obstacle_list and temp_x >= 0 and temp_x < len(grid) and temp_y >= 0 and temp_y < len(grid[0]) and closed_grid[temp_x][temp_y] == 0:
                    
                    closed_grid[temp_x][temp_y] = 1
                    
                    action[temp_x][temp_y] = i
                    
                    open_list.append([temp_g, temp_x, temp_y])
    
    #### track the optimal path in a reversed way
    x = goal[0]
    y = goal[1]
    
    action_cell = [[' ' for row in range(len(grid[0]))] for col in range(len(grid))]
    action_cell[x][y] = '*'
    
    while x != init[0] or y != init[1]:
        
        x2 = x - delta[action[x][y]][0]
        y2 = y - delta[action[x][y]][1]
        
        action_cell[x2][y2] = delta_name[action[x][y]]
        
        x = x2
        y = y2    
    
    expand = expand_grid
    return path, expand, action_cell

In [5]:
path, expand, action_cell = search(grid,init,goal,cost)

print('optimal cost: ',path)


print('the explored steps in the map grid:', expand)


print('showing the optimal path')
action_cell

optimal cost:  [9, 4, 5]
the explored steps in the map grid: [[0, 2, -1, 12, 15, 18], [1, 4, 7, 10, 14, 17], [3, 6, -1, 13, -1, 20], [5, 9, -1, 16, -1, 21], [8, 11, -1, 19, -1, 22]]
showing the optimal path


[['v', ' ', ' ', ' ', ' ', ' '],
 ['>', '>', '>', '>', '>', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', '*']]

In [6]:
grid = [[0, 1, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 1],
        [0, 0, 0, 1, 0, 0, 0]]
init = [0,0]
goal = [len(grid)-1,len(grid[0])-1]
cost = 1

In [7]:
path, expand, action_cell = search(grid,init,goal,cost)

print('optimal cost: ',path)


print('the explored steps in the map grid:', expand)


print('showing the optimal path')
action_cell

optimal cost:  [18, 4, 6]
the explored steps in the map grid: [[0, -1, 10, 11, 12, -1, -1], [1, -1, 9, -1, 13, -1, -1], [2, -1, 8, -1, 14, -1, -1], [3, -1, 7, -1, 15, -1, -1], [4, 5, 6, -1, 16, 17, 18]]
showing the optimal path


[['v', ' ', '>', '>', 'v', ' ', ' '],
 ['v', ' ', '^', ' ', 'v', ' ', ' '],
 ['v', ' ', '^', ' ', 'v', ' ', ' '],
 ['v', ' ', '^', ' ', 'v', ' ', ' '],
 ['>', '>', '^', ' ', '>', '>', '*']]

## A* search (with heuristic function)
By the guidance of predefined heuristic function, such as Euclidean distance to the goal, the optimal path search algorithm can be more efficient. Note the heuristic function $f$ should underestimate the actual cost to the goal, for function $f=0$ (without heuristic information), the A* search turns into the above search algorithm.

In [8]:
# -----------
# User Instructions:
#
# Modify the the search function so that it becomes
# an A* search algorithm as defined in the previous
# lectures.
#
# Your function should return the expanded grid
# which shows, for each element, the count when
# it was expanded or -1 if the element was never expanded.
# 
# If there is no path from init to goal,
# the function should return the string 'fail'
# ----------

grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]]
heuristic = [[9, 8, 7, 6, 5, 4],
             [8, 7, 6, 5, 4, 3],
             [7, 6, 5, 4, 3, 2],
             [6, 5, 4, 3, 2, 1],
             [5, 4, 3, 2, 1, 0]]

init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']





In [11]:
def search(grid,init,goal,cost,heuristic):
    # ----------------------------------------
    # modify the code below
    # ----------------------------------------
    closed = [[0 for col in range(len(grid[0]))] for row in range(len(grid))]
    closed[init[0]][init[1]] = 1

    expand = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    action = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    
    
    x = init[0]
    y = init[1]
    g = 0
    h = heuristic[x][y]

    open = [[g+h, x, y]]

    found = False  # flag that is set when search is complete
    resign = False # flag set if we can't find expand
    count = 0
    
    while not found and not resign:
        if len(open) == 0:
            resign = True
            return "Fail"
        else:
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[1]
            y = next[2]
            gh = next[0]
            expand[x][y] = count
            count += 1
            
            if x == goal[0] and y == goal[1]:
                found = True
            else:
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >=0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            gh2 = g + cost + heuristic[x2][y2]
                            open.append([gh2, x2, y2])
                            closed[x2][y2] = 1
                            action[x2][y2] = i
                            
    
    #### track the optimal path in a reversed way
    x = goal[0]
    y = goal[1]
    
    action_cell = [[' ' for row in range(len(grid[0]))] for col in range(len(grid))]
    action_cell[x][y] = '*'
    
    while x != init[0] or y != init[1]:
        
        x2 = x - delta[action[x][y]][0]
        y2 = y - delta[action[x][y]][1]
        
        action_cell[x2][y2] = delta_name[action[x][y]]
        
        x = x2
        y = y2    
    
    return path, expand, action_cell


In [12]:
_,expand, action_cell = search(grid,init,goal,cost,heuristic)

print('the explored steps in the map grid:', expand)

print('showing the optimal path')
action_cell

the explored steps in the map grid: [[0, -1, -1, -1, -1, -1], [1, -1, -1, -1, -1, -1], [2, -1, -1, -1, -1, -1], [3, -1, -1, 8, 9, 10], [4, 5, 6, 7, -1, 11]]
showing the optimal path


[['v', ' ', ' ', ' ', ' ', ' '],
 ['v', ' ', ' ', ' ', ' ', ' '],
 ['v', ' ', ' ', ' ', ' ', ' '],
 ['v', ' ', ' ', '>', '>', 'v'],
 ['>', '>', '>', '^', ' ', '*']]

## Dynamic programming
Given the goal position, dynamic programming can provide the optimal costs for individual positions to the goal. It does not necessarily need the initialization point, starting from any point can lead to the goal position. 

$f(x,y)=\min_{x',y'}f(x',y')+1$, where $f(x',y')$ is the optimal neighbor and 1 is the update cost of each step.

In [8]:
# ----------
# User Instructions:
# 
# Implement the function optimum_policy2D below.
#
# You are given a car in grid with initial state
# init. Your task is to compute and return the car's 
# optimal path to the position specified in goal; 
# the costs for each motion are as defined in cost.
#
# There are four motion directions: up, left, down, and right.
# Increasing the index in this array corresponds to making a
# a left turn, and decreasing the index corresponds to making a 
# right turn.

forward = [[-1,  0], # go up
           [ 0, -1], # go left
           [ 1,  0], # go down
           [ 0,  1]] # go right
forward_name = ['up', 'left', 'down', 'right']

# action has 3 values: right turn, no turn, left turn
action = [-1, 0, 1]
action_name = ['R', '#', 'L']

# EXAMPLE INPUTS:
# grid format:
#     0 = navigable space
#     1 = unnavigable space 
grid = [[1, 1, 1, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1, 1]]

init = [4, 3, 0] # given in the form [row,col,direction]
                 # direction = 0: up
                 #             1: left
                 #             2: down
                 #             3: right
                
goal = [2, 0] # given in the form [row,col]

cost = [2, 1, 20] # cost has 3 values, corresponding to making 
                  # a right turn, no turn, and a left turn

# grid = [[0, 0, 0, 0, 1, 1],
#         [0, 0, 1, 0, 0, 0],
#         [0, 0, 0, 0, 1, 0],
#         [0, 0, 1, 1, 1, 0],
#         [0, 0, 0, 0, 1, 0]]
# init = [4, 5, 0]
# goal = [4, 3]
# cost = [1, 1, 1]
# EXAMPLE OUTPUT:
# calling optimum_policy2D with the given parameters should return 
# [[' ', ' ', ' ', 'R', '#', 'R'],
#  [' ', ' ', ' ', '#', ' ', '#'],
#  ['*', '#', '#', '#', '#', 'R'],
#  [' ', ' ', ' ', '#', ' ', ' '],
#  [' ', ' ', ' ', '#', ' ', ' ']]
# ----------

In [9]:
def optimum_policy2D(grid,init,goal,cost):

    
    value = [[[999 for row in range(len(grid[0]))] for col in range(len(grid))],
             [[999 for row in range(len(grid[0]))] for col in range(len(grid))],
             [[999 for row in range(len(grid[0]))] for col in range(len(grid))],
             [[999 for row in range(len(grid[0]))] for col in range(len(grid))]]
    
    
    policy = [[[' ' for row in range(len(grid[0]))] for col in range(len(grid))],
             [[' ' for row in range(len(grid[0]))] for col in range(len(grid))],
             [[' ' for row in range(len(grid[0]))] for col in range(len(grid))],
             [[' ' for row in range(len(grid[0]))] for col in range(len(grid))]]
    
    
    
    change = True
    
    
    while change:
        change = False
        
        
        for x in range(len(grid)):
            for y in range(len(grid[0])):
                for d in range(len(value)):
                    if goal[0] == x and goal[1] == y:
                        if value[d][x][y] > 0:
                            value[d][x][y] = 0

                            policy[d][x][y] = '*'

                            change = True

                    elif grid[x][y] == 0:

                        for idx, act in enumerate(action):

                            d2 = (d + act) % len(forward)
                            x2 = x + forward[d2][0]
                            y2 = y + forward[d2][1]

                            if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]) and grid[x2][y2] == 0:
                                v2 = value[d2][x2][y2] + cost[idx]

                                if v2 < value[d][x][y]:
                                    change = True
                                    value[d][x][y] = v2
                                    policy[d][x][y] = action_name[idx]
    
    policy2 = [[' ' for row in range(len(grid[0]))] for col in range(len(grid))]
    
    d = init[2]
    x = init[0]
    y = init[1]
    
    policy2[x][y] = policy[d][x][y]
    while policy[d][x][y] != '*':
        if policy[d][x][y] == '#':
            d2 = d
        elif policy[d][x][y] == 'R':
            d2 = (d-1)%4
        elif policy[d][x][y] == 'L':
            d2 = (d+1)%4
        x = x + forward[d2][0]
        y = y + forward[d2][1]
        d = d2
        
        policy2[x][y] = policy[d][x][y]
        
    return policy, policy2
                                
                                
                    
                    

In [10]:
policy, policy2 = optimum_policy2D(grid,init,goal,cost)

policy2

[[' ', ' ', ' ', 'R', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', '#'],
 ['*', '#', '#', '#', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', ' '],
 [' ', ' ', ' ', '#', ' ', ' ']]